In [1]:
from glob import glob as gg
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm as cm
from matplotlib import rc as rc
#
from netCDF4 import Dataset 
#
from scipy.io import loadmat
from scipy.signal import find_peaks as fp
from scipy.interpolate import interp1d
#
import pandas as pd
#
from datetime import datetime
#
import gsw

In [3]:
#
# load time properties
#
class REcoM_time:
    def __init__(self):
        # load time properties
        self.estimate_time_axis()
    def estimate_time_axis(self):
        path_obs = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/config/'
        filename = path_obs + 'time.recom'
        # read data 
        with open(filename) as f:
            lines = f.readlines()
        for l in lines:
            tmp = l.replace(' ','').replace('\t','').split('=')[-1].strip()
            if 'start_date' in l:
                date_start = datetime.strptime(tmp,'%d-%m-%Y').toordinal()
            elif 'end_date' in l:
                date_end = datetime.strptime(tmp,'%d-%m-%Y').toordinal()
            elif 'dt' in l:
                dt = float(tmp)/24.
            elif 'spinup' in l:
                spinup = int(tmp)
        self.date_start, self.date_end = np.copy(date_start), np.copy(date_end)
        date_start = date_start - spinup
        
        # create time axis
        npt = int(abs(date_end - date_start)/dt)
        self.dates = np.linspace(date_start, date_end, npt+1)        
#
time = REcoM_time()

In [23]:
class MSS_data(object):
    def __init__(self, time):
        # load MSS daily data
        self.Process_MSS_data(time)
        
    def Process_MSS_data(self, time):
        path_obs = '/home/fbirrien/Data/Observations/MOSAiC/ocean/'
        filename = path_obs + 'MOSAiC_MSS_daily_avg_Kz.nc'
        
        # read data
        ncid = Dataset(filename, "r", format="NETCDF4")
        
        # dates
        dates = np.floor(ncid['stime'][:])-366
        
        # crop according to simulation time
        indices = list(set(np.where(dates>=time.date_start)[0]) & set(np.where(dates<=time.date_end)[0]))
        
        self.dates = dates[indices]
        
        # latitude and longitude
        longitude, latitude = ncid['slongitude'][:], ncid['slatitude'][:]
        self.longitude, self.latitude = longitude[indices], latitude[indices] 
        
        # pressure
        pressure = ncid['press'][:]
        self.pressure = pressure[:,indices]

        # salinity (potential)
        salinity = ncid['psal'][:]
        self.salinity = salinity[:,indices]
        
        # temperature (potential)
        temperature = ncid['ptemp'][:]
        self.temperature = temperature[:,indices]
        
        # density (in_situ)
        density = ncid['rho'][:]
        self.density = density[:,indices]
        
        # vertical diffusivity Kz 
        Kz = ncid['krho'][:]
        self.Kz = Kz[:,indices]
        ncid.close()
#
mss_data = MSS_data(time)

In [24]:
#
# load vertical grid
#
class REcoM_grid:
    def __init__(self):
        # load time properties
        self.read_grid_data()
    def read_grid_data(self):
        path_grid = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/grid/'
        filename = path_grid + 'REcoM1D_mesh.nc'
        
        # read data
        ncid = Dataset(filename, "r", format="NETCDF4")
        # dates
        self.dates = ncid['dates'][:]
        
        # vertical discretization
        self.z, self.nlevels = ncid['Z'][:], ncid['nlevels'][:]
#
vertical_grid = REcoM_grid()

In [80]:
#
# convert MSS data and interpolate data on the vertical axis
#
class Vertical_Interpolation:
    def __init__(self, mss_data, vertical_grid):
        # load time properties
        self.vertical_interpolation(mss_data, vertical_grid)
        
    def vertical_interpolation(self, mss_data, vertical_grid):
        # initialization
        var = mss_data
        salinity, temperature, density, Kz = [], [], [], []
        nz = len(vertical_grid.z)
        # crop MSS data to simulation time requirement
        for i, dt in enumerate(var.dates):
            # convert pressure axis into depth
            P, lat = np.asarray(var.pressure[:,i]), var.latitude[i]
            iP = np.where(P>=0)
            P = P[iP]
            z = gsw.z_from_p(P,lat)
    
            # look for corresponding mesh profiles
            ind = np.argmin(abs(dt-vertical_grid.dates))
            nlvl = int(vertical_grid.nlevels[ind])
            zv = np.asarray(vertical_grid.z[0:nlvl])
    
            # initialization of the temperature, salinity, density profiles
            sal, temp, dens, K = np.zeros(nz)+np.nan, np.zeros(nz)+np.nan, np.zeros(nz)+np.nan, np.zeros(nz)+np.nan
    
            # interpolate variable on REcoM grid
            ind1, ind2 = np.where(zv>=np.min(z))[0], np.where(zv<np.min(z))[0] 
    
            # salinity
            vs=np.reshape(var.salinity[iP,i], var.salinity[iP,i].size)
            f = interp1d(z, vs, kind='linear')
            sal[ind1]=f(zv[ind1])
            sal[ind2] = sal[ind1[-1]]
    
            # temperature
            vt=np.reshape(var.temperature[iP,i], var.temperature[iP,i].size)
            f = interp1d(z, vt, kind='linear')
            temp[ind1]=f(zv[ind1])
            temp[ind2] = temp[ind1[-1]]
    
            # density
            vd=np.reshape(var.density[iP,i], var.density[iP,i].size)
            f = interp1d(z, vd, kind='linear')
            dens[ind1]=f(zv[ind1])
            dens[ind2]=dens[ind1[-1]]    
    
            # Kz
            vk=np.reshape(var.Kz[iP,i], var.Kz[iP,i].size)
            f = interp1d(z, vk, kind='linear')
            K[ind1]=f(zv[ind1])
            K[ind2] = K[ind1[-1]]
    
            # store data
            salinity.append(sal), temperature.append(temp), density.append(dens), Kz.append(K)

        # store data
        self.dates, self.z, self.nlevels =  mss_data.dates, vertical_grid.z, vertical_grid.nlevels
        self.longitude, self.latitude = mss_data.longitude, mss_data.latitude
        salinity, temperature = np.asarray(salinity), np.asarray(temperature)
        density, Kz = np.asarray(density), np.asarray(Kz)
        # special processing after npasarray
        salinity[salinity<-1e3], temperature[temperature<-1e3]  = np.nan, np.nan
        density[density<-1e3], Kz[Kz<1e3] = np.nan, np.nan
        # 
        self.salinity, self.temperature = salinity, temperature
        self.density, self.Kz = density, Kz
#
data = Vertical_Interpolation(mss_data, vertical_grid)

In [ ]:
#remark::
# at this stage there are multiple vertical profiles with NaN values (in the nlevels of computation), can we extrapolate the data to fill the gap?

In [ ]:
#
# interpolate MSS data in time according to REcoM time discretization
#